In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Dropout
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [4]:
print(os.listdir("/gdrive/My Drive/DerinÖğrenme/Pneumonia/X-Ray"))

['Bacterial', 'Viral']


In [5]:
DATADIR = '/gdrive/My Drive/DerinÖğrenme/Pneumonia/X-Ray'
CATEGORIES = ["Bacterial","Viral"]

In [6]:
IMG_SIZE = 250

In [8]:
import cv2

all_data =[]

def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                pass
create_all_data()

In [9]:
print("Total number of images: ", len(all_data))

Total number of images:  16167


In [10]:
X = []
y = []

for categories, label in all_data:
  X.append(categories)
  y.append(label)

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [12]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [13]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(11639, 250, 250, 3)
(3234, 250, 250, 3)
(1294, 250, 250, 3)
(11639,)
(3234,)
(1294,)


In [14]:
import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [15]:
model = Sequential()

In [16]:
from keras import regularizers
model.add(Flatten(input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(2, activation='softmax'))

In [17]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 187500)            0         
                                                                 
 dense (Dense)               (None, 128)               24000128  
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 batch_normalization (Batch  (None, 32)                128       
 Normalization)                                                  
                                                                 
 dense_3 (Dense)             (None, 2)                 66        
                                                        

In [19]:
from tensorflow import keras
# En iyi performans gösteren modelin ağırlıkları kaydeder.
callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        # Sadece en iyi modeli kaydet
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),

    keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=3)
]

In [20]:
history = model.fit(x_train, train_yCl, epochs=10, validation_data=(x_val, valid_yCl), callbacks=callback_list,)

Epoch 1/10
364/364 [==============================] - ETA: 0s - loss: 2.7075 - accuracy: 0.7121
Epoch 1: val_accuracy improved from -inf to 0.60974, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


364/364 [==============================] - 66s 177ms/step - loss: 2.7075 - accuracy: 0.7121 - val_loss: 1.0455 - val_accuracy: 0.6097
Epoch 2/10
364/364 [==============================] - ETA: 0s - loss: 0.8126 - accuracy: 0.7188
Epoch 2: val_accuracy did not improve from 0.60974
364/364 [==============================] - 64s 175ms/step - loss: 0.8126 - accuracy: 0.7188 - val_loss: 1.8387 - val_accuracy: 0.6090
Epoch 3/10
364/364 [==============================] - ETA: 0s - loss: 0.8135 - accuracy: 0.7181
Epoch 3: val_accuracy improved from 0.60974 to 0.61824, saving model to model.h5
364/364 [==============================] - 64s 177ms/step - loss: 0.8135 - accuracy: 0.7181 - val_loss: 0.8658 - val_accuracy: 0.6182
Epoch 4/10
364/364 [==============================] - ETA: 0s - loss: 0.8322 - accuracy: 0.7087
Epoch 4: val_accuracy did not improve from 0.61824
364/364 [==============================] - 63s 173ms/step - loss: 0.8322 - accuracy: 0.7087 - val_loss: 3.0401 - val_accuracy: 

In [21]:
test_loss, test_acc = model.evaluate(x_test, test_yCl)

102/102 [==============================] - 2s 17ms/step - loss: 18.2902 - accuracy: 0.3788


In [22]:
# Test seti üzerinde modelin performansını değerlendirme
score_test = model.evaluate(x_test, test_yCl)
print("Accuracy: ", score_test[1])

102/102 [==============================] - 2s 18ms/step - loss: 18.2902 - accuracy: 0.3788
Accuracy:  0.3787878751754761


In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

def evaluate_model(model, x_test, y_test):
    # Accuracy Score
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(test_yCl, axis=1)
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    print(f"Accuracy: {accuracy}")

    # Precision Score
    precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
    print(f"Precision: {precision}")

    # Recall Score
    recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
    print(f"Recall: {recall}")

    # F1 Score
    f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')
    print(f"F1-Score: {f1}")

    # Classification Report
    print("Classification Report:")
    print(classification_report(y_test_classes, y_pred_classes))


# Modeli değerlendir
evaluate_model(model, x_test, y_test)

102/102 [==============================] - 2s 18ms/step
Accuracy: 0.3787878787878788
Precision: 0.14348025711662074
Recall: 0.3787878787878788
F1-Score: 0.2081252081252081
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2009
           1       0.38      1.00      0.55      1225

    accuracy                           0.38      3234
   macro avg       0.19      0.50      0.27      3234
weighted avg       0.14      0.38      0.21      3234



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13